In [1]:
import numpy as np
import pandas as pd
import altair as alt

# Save a vega-lite spec and a PNG blob for each plot in the notebook
alt.renderers.enable("mimetype")
# Handle large data sets without embedding them in the notebook
alt.data_transformers.enable("data_server");

In [2]:
stinson2019 = pd.read_pickle("../data/stinson2019/stinson2019_df")
HF_weather = pd.read_csv('../data/HF_weather/raw/hf001-08-hourly-m.csv', parse_dates=['datetime'])

/home/steffen/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (3,5,7,9,11,13,15,17,19,21,23,25,27,29) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
# stinson2019_df["jd"] = pd.DatetimeIndex(stinson2019_df["date"]).dayofyear
# stinson2019_df["year_int"] = pd.DatetimeIndex(stinson2019_df["year"]).year
# stinson2019_df["sugar_wt"] = stinson2019_df["sugar"]/100 * stinson2019_df["sap_wt"]
# stinson2019_df["yearly_sap"] = np.nan
# stinson2019_df["yearly_sugar"] = np.nan
# stinson2019_df['tree_tap'] = stinson2019_df['tree']+ '_' + stinson2019_df['tap']
# stinson2019_df = stinson2019_df.sort_values(by="date")

# abs_measurement = 0

# for year in stinson2019_df["year_int"].unique():
#     year_df = stinson2019_df[stinson2019_df["year_int"] == year]

#     for site in stinson2019_df[stinson2019_df["year_int"] == year]["site"].unique():
#         for tree in stinson2019_df[
#             (stinson2019_df["year_int"] == year) & (stinson2019_df["site"] == site)
#         ]["tree"].unique():
#             for tap in stinson2019_df[
#                 (stinson2019_df["year_int"] == year)
#                 & (stinson2019_df["site"] == site)
#                 & (stinson2019_df["tree"] == tree)
#             ]["tap"].unique():
#                 tap_yr_index = stinson2019_df.index[
#                     (stinson2019_df["year_int"] == year)
#                     & (stinson2019_df["site"] == site)
#                     & (stinson2019_df["tree"] == tree)
#                     & (stinson2019_df["tap"] == tap)
#                 ].tolist()
#                 yearly_sap = 0
#                 yearly_sugar = 0
#                 for measurement in tap_yr_index:
#                     yearly_sap = np.nansum(
#                         [yearly_sap, stinson2019_df.loc[measurement, "sap_wt"]]
#                     )
#                     yearly_sugar = np.nansum(
#                         [
#                             yearly_sugar,
#                             stinson2019_df.loc[measurement, "sap_wt"]
#                             * stinson2019_df.loc[measurement, "sugar"]/100,
#                         ]
#                     )
#                     stinson2019_df.loc[measurement, "yearly_sap"] = yearly_sap
#                     stinson2019_df.loc[measurement, "yearly_sugar"] = yearly_sugar


# #     yearly_sap = 0
# #     yearly_sugar = 0
# #     for measurement in list(range(len(year_df))):
# #         yearly_sap+=stinson2019_df.iloc[abs_measurement,6]
# #         yearly_sugar+=stinson2019_df.iloc[abs_measurement,6]*stinson2019_df.iloc[abs_measurement,5]
# #         stinson2019_df.iloc[abs_measurement,13]=yearly_sap
# #         stinson2019_df.iloc[abs_measurement,14]=yearly_sugar
# #         abs_measurement+=1

In [23]:
stinson2019 = stinson2019.sort_values(["site", "tree", "tap", "date"])

# Create unique record ids for each entry
stinson2019["record_id"] = (
    + stinson2019["tree"]
    + "_"
    + stinson2019["tap"]
    + "_"
    + pd.DatetimeIndex(stinson2019["year"]).year.astype(str)
    + "_"
)

for tapyear in stinson2019["record_id"].unique():
    stinson2019.loc[stinson2019["record_id"] == tapyear, "record_id"] += [
        str(i)
        for i in range(stinson2019[stinson2019["record_id"] == tapyear].shape[0])
    ]
    
stinson2019['tap_id'] = stinson2019['tree']+stinson2019['tap']


# normalize data tables
tap_records = stinson2019[['record_id', 'tap_id']].set_index('record_id')
sap = stinson2019[['record_id', 'sap_wt']].rename(columns={"sap_wt": "sap"}).set_index('record_id')
sugar = stinson2019[['record_id', 'sugar']].set_index('record_id')
dates = stinson2019[['record_id', 'date']].set_index('record_id')
dates.loc[:,'date'] = pd.DatetimeIndex(dates['date'])
tap_tree = stinson2019[['tap_id','tree']].drop_duplicates().set_index('tap_id')
species = stinson2019[['tree','species']].drop_duplicates().set_index('tree')
site = stinson2019[['tree','site']].drop_duplicates().set_index('tree')

In [5]:
stinson2019

,site,date,tree,species,tap,sugar,sap_wt,year,site_id,tree_id,tree_record_id,tap_record_id,record_id,tap_id
5299,Dartmouth Organic Farm,2014-03-11,DOF1,ACSA,A,1.2,0.03,2014-01-01,DOF,DOF.ACSA.DOF1,DOF.ACSA.DOF1.2014-03-11,DOF.ACSA.DOF1.A.2014-03-11,DOF1_A_2014_0,DOF1A
5322,Dartmouth Organic Farm,2014-03-17,DOF1,ACSA,A,NaN,0.01,2014-01-01,DOF,DOF.ACSA.DOF1,DOF.ACSA.DOF1.2014-03-17,DOF.ACSA.DOF1.A.2014-03-17,DOF1_A_2014_1,DOF1A
5345,Dartmouth Organic Farm,2014-03-30,DOF1,ACSA,A,NaN,NaN,2014-01-01,DOF,DOF.ACSA.DOF1,DOF.ACSA.DOF1.2014-03-30,DOF.ACSA.DOF1.A.2014-03-30,DOF1_A_2014_2,DOF1A
5368,Dartmouth Organic Farm,2014-03-31,DOF1,ACSA,A,NaN,0.01,2014-01-01,DOF,DOF.ACSA.DOF1,DOF.ACSA.DOF1.2014-03-31,DOF.ACSA.DOF1.A.2014-03-31,DOF1_A_2014_3,DOF1A
5391,Dartmouth Organic Farm,2014-04-01,DOF1,ACSA,A,2.2,0.67,2014-01-01,DOF,DOF.ACSA.DOF1,DOF.ACSA.DOF1.2014-04-01,DOF.ACSA.DOF1.A.2014-04-01,DOF1_A_2014_4,DOF1A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7785,Southernmost Maple,2016-02-28,SMAS8,ACSA,B,2.1,15.11,2016-01-01,SMM,SMM.ACSA.SMAS8,SMM.ACSA.SMAS8.2016-02-28,SMM.ACSA.SMAS8.B.2016-02-28,SMAS8_B_2016_3,SMAS8B
7817,Southernmost Maple,2016-02-29,SMAS8,ACSA,B,1.8,9.11,2016-01-01,SMM,SMM.ACSA.SMAS8,SMM.ACSA.SMAS8.2016-02-29,SMM.ACSA.SMAS8.B.2016-02-29,SMAS8_B_2016_4,SMAS8B
7849,Southernmost Maple,2016-03-02,SMAS8,ACSA,B,2.4,10.41,2016-01-01,SMM,SMM.ACSA.SMAS8,SMM.ACSA.SMAS8.2016-03-02,SMM.ACSA.SMAS8.B.2016-03-02,SMAS8_B_2016_5,SMAS8B
7881,Southernmost Maple,2016-03-06,SMAS8,ACSA,B,1.8,17.01,2016-01-01,SMM,SMM.ACSA.SMAS8,SMM.ACSA.SMAS8.2016-03-06,SMM.ACSA.SMAS8.B.2016-03-06,SMAS8_B_2016_6,SMAS8B


In [30]:
50//7

7

In [28]:
df = tap_records.join(tap_tree.iloc[[0]], how="right", on="tap_id").join(
    sap, how="left").join(
    sugar, how="left").join(
    dates, how="left")
df['year'] = pd.DatetimeIndex(df['date']).year
df['jd'] = pd.DatetimeIndex(df['date']).dayofyear
for year in df['year']:
    df_year = df[df['year']==year]
    num_wks = df_year['jd'].max()//7
    weeks = list(range(0,num_wks,7))
    df.groupby(df_year]['date']).sum()

Int64Index([2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014,
            2014, 2014, 2014, 2014, 2014, 2014, 2014, 2015, 2015, 2015, 2015,
            2015, 2015, 2015, 2015, 2015, 2015, 2015, 2015, 2015, 2015, 2015,
            2015, 2015, 2016, 2016, 2016, 2016, 2016, 2016, 2016, 2016, 2016,
            2016, 2016, 2016, 2016, 2016, 2016, 2016, 2016, 2016, 2016, 2016,
            2016, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017,
            2017, 2017],
           dtype='int64', name='date')

In [ ]:
def get_weekly_data(location = 'all', tree= 'all',tap_id='all',year='all'):
    df = tap_records.join(tap_tree.iloc[[0]], how="right", on="tap_id").join(
        sap, how="left").join(
        sugar, how="left").join(
        dates, how="left")
    df['year'] = pd.DatetimeIndex(df['date']).year
    df['jd'] = pd.DatetimeIndex(df['date']).dayofyear
    for year in df['year']:
        df_year = df[df['year']==year]
        num_wks = df_year['jd'].max()//7
        weeks = list(range(0,num_wks,7))
        df.groupby(df_year]['date']).sum()
        harvard_sap.groupby([harvard_sap['year'], pd.cut(df_year['jd'], weeks)]).sum()
    if year == 'all':
        years = 
    
    weekly_df = 'tap_id', 'dates', 'year', 'week', 'c_sap', 'c_sugar', 'fr_thw', 'gdd'
    
    return weekly_df

In [98]:
stinson2019

,site,date,tree,species,tap,sugar,sap_wt,year,site_id,tree_id,tree_record_id,tap_record_id,record_id,tap_id
5299,Dartmouth Organic Farm,2014-03-11,DOF1,ACSA,A,1.2,0.03,2014-01-01,DOF,DOF.ACSA.DOF1,DOF.ACSA.DOF1.2014-03-11,DOF.ACSA.DOF1.A.2014-03-11,DOF1_A_2012_0,DOF1A
5322,Dartmouth Organic Farm,2014-03-17,DOF1,ACSA,A,NaN,0.01,2014-01-01,DOF,DOF.ACSA.DOF1,DOF.ACSA.DOF1.2014-03-17,DOF.ACSA.DOF1.A.2014-03-17,DOF1_A_2012_1,DOF1A
5345,Dartmouth Organic Farm,2014-03-30,DOF1,ACSA,A,NaN,NaN,2014-01-01,DOF,DOF.ACSA.DOF1,DOF.ACSA.DOF1.2014-03-30,DOF.ACSA.DOF1.A.2014-03-30,DOF1_A_2012_2,DOF1A
5368,Dartmouth Organic Farm,2014-03-31,DOF1,ACSA,A,NaN,0.01,2014-01-01,DOF,DOF.ACSA.DOF1,DOF.ACSA.DOF1.2014-03-31,DOF.ACSA.DOF1.A.2014-03-31,DOF1_A_2012_3,DOF1A
5391,Dartmouth Organic Farm,2014-04-01,DOF1,ACSA,A,2.2,0.67,2014-01-01,DOF,DOF.ACSA.DOF1,DOF.ACSA.DOF1.2014-04-01,DOF.ACSA.DOF1.A.2014-04-01,DOF1_A_2012_4,DOF1A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7785,Southernmost Maple,2016-02-28,SMAS8,ACSA,B,2.1,15.11,2016-01-01,SMM,SMM.ACSA.SMAS8,SMM.ACSA.SMAS8.2016-02-28,SMM.ACSA.SMAS8.B.2016-02-28,SMAS8_B_2017_3,SMAS8B
7817,Southernmost Maple,2016-02-29,SMAS8,ACSA,B,1.8,9.11,2016-01-01,SMM,SMM.ACSA.SMAS8,SMM.ACSA.SMAS8.2016-02-29,SMM.ACSA.SMAS8.B.2016-02-29,SMAS8_B_2017_4,SMAS8B
7849,Southernmost Maple,2016-03-02,SMAS8,ACSA,B,2.4,10.41,2016-01-01,SMM,SMM.ACSA.SMAS8,SMM.ACSA.SMAS8.2016-03-02,SMM.ACSA.SMAS8.B.2016-03-02,SMAS8_B_2017_5,SMAS8B
7881,Southernmost Maple,2016-03-06,SMAS8,ACSA,B,1.8,17.01,2016-01-01,SMM,SMM.ACSA.SMAS8,SMM.ACSA.SMAS8.2016-03-06,SMM.ACSA.SMAS8.B.2016-03-06,SMAS8_B_2017_6,SMAS8B


In [7]:
harvard_sap = stinson2019_df[stinson2019_df['site_id']=='HF']
harvard_sap['jd'].max()

102

In [8]:
weeks = list(range(0,106,7))
harvard_sap.groupby([harvard_sap['year'], pd.cut(harvard_sap['jd'], weeks)]).sum()

sugar  sap_wt       jd  year_int  sugar_wt  yearly_sap  \
year       jd                                                                  
2012-01-01 (0, 7]       NaN     NaN      NaN       NaN       NaN         NaN   
           (7, 14]      NaN     NaN      NaN       NaN       NaN         NaN   
           (14, 21]     NaN     NaN      NaN       NaN       NaN         NaN   
           (21, 28]     NaN     NaN      NaN       NaN       NaN         NaN   
           (28, 35]     NaN     NaN      NaN       NaN       NaN         NaN   
...                     ...     ...      ...       ...       ...         ...   
2017-01-01 (70, 77]     NaN     NaN      NaN       NaN       NaN         NaN   
           (77, 84]   196.8  287.54   7632.0  193632.0   6.73874     4241.60   
           (84, 91]   498.2  823.22  20880.0  484080.0  18.96144    13784.15   
           (91, 98]   243.8  343.14  12300.0  266244.0   7.71448     9610.41   
           (98, 105]    NaN     NaN      NaN       NaN       NaN         NaN   

                      yearly_sugar  
year       jd                       
2012-01-01 (0, 7]              NaN  
           (7, 14]             NaN  
           (14, 21]            NaN  
           (21, 28]            NaN  
           (28, 35]            NaN  
...                            ...  
2017-01-01 (70, 77]            NaN  
           (77, 84]      103.54440  
           (84, 91]      332.57985  
           (91, 98]      231.82877  
           (98, 105]           NaN  

[90 rows x 7 columns]

In [9]:
pd.periodindex

AttributeError: module 'pandas' has no attribute 'periodindex'